#  Cats and dogs recognition

## Učitavanje biblioteka

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import time

In [ ]:
#funkcija za računanje metrika
def metrike(modeli, testData, y_test, time):
    for ind, model in enumerate(modeli):
        print(F"Algoritam: {type(model.optimizer).__name__}")
        #ako je ANN
        y_pred = model.predict(testData)
        y_pred = (y_pred > 0.5)
        print("Accuracy: ", metrics.accuracy_score(y_test, y_pred))
        print('ROC AUC:', metrics.roc_auc_score(y_test, y_pred))
        print("Confusion matrix")
        CM = metrics.confusion_matrix(y_test, y_pred) #confusion matrica
        TN = CM[0, 0]
        TP = CM[1, 1]
        FP = CM[0, 1]
        FN = CM[1, 0]
        print("    P0           P1")
        print(f"S0  {TN}        {FP}")
        print(f"S1  {FN}        {TP}")
        print("Recall: ",metrics.recall_score(y_test,y_pred))
        print("Precision: ",  metrics.precision_score(y_test, y_pred))
        print("F1 score: ", metrics.f1_score(y_test, y_pred))
        print(f"Training time (sec): {time[ind]}" )
        print("-----------------------------------------------")
def metrike_table(modeli, testData, y_test, tr_time, histories):
    data = []
    for ind, model in enumerate(modeli):
        start = time.time()
        y_pred = model.predict(testData)
        y_pred = (y_pred > 0.5)
        test_time = time.time()-start
        data.append([
            type(model.optimizer).__name__,
            metrics.accuracy_score(y_test, y_pred),
            metrics.roc_auc_score(y_test, y_pred),
            metrics.recall_score(y_test,y_pred),
            metrics.precision_score(y_test, y_pred),
            metrics.f1_score(y_test, y_pred),
            tr_time[ind],
            test_time,
            len(histories[ind].history['loss'])
            ])
        plt.plot(pd.DataFrame(histories[ind].history["accuracy"]))
        plt.plot(pd.DataFrame(histories[ind].history["val_accuracy"]))
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.title(f'{type(model.optimizer).__name__} accuracy')
        plt.figure(figsize=(6,6), dpi=500)
        plt.show()

        plt.plot(pd.DataFrame(histories[ind].history["loss"]))
        plt.plot(pd.DataFrame(histories[ind].history["val_loss"]))
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.title(f'{type(model.optimizer).__name__} loss')
        plt.figure(figsize=(6,6), dpi=500)
        plt.show()
    df = pd.DataFrame(data, columns = ['Algoritam', "Accuracy",'ROC AUC',"Recall","Precision","F1 score", "Training time (sec)","Test time (sec)",'Epochs'])
    return df

## Učitavanje skupa podataka

In [ ]:
datagen = ImageDataGenerator(rescale = 1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
X_train = datagen.flow_from_directory('../datasets/cats_and_dogs/train_images', color_mode='rgb',target_size = (64, 64),batch_size = 64,class_mode = 'binary', seed=42)

In [ ]:
datagen = ImageDataGenerator(rescale = 1./255)
x_test = datagen.flow_from_directory('../datasets/cats_and_dogs/test_images', color_mode='rgb', shuffle=False, target_size = (64, 64),batch_size = 64,class_mode = 'binary', seed=42)


## CNN - ADAM

In [ ]:
cnn_adam = tf.keras.Sequential()
#tri konvolucijska sa po 32, 64 i 128 filtera i 3 sloja sažimanja
cnn_adam.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
cnn_adam.add(tf.keras.layers.MaxPooling2D((2, 2)))
cnn_adam.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
cnn_adam.add(tf.keras.layers.MaxPooling2D((2, 2)))
cnn_adam.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
cnn_adam.add(tf.keras.layers.MaxPooling2D((2, 2)))
#sloj za spljoštavanje mape
cnn_adam.add(tf.keras.layers.Flatten())
#dva skrivena sloja sa 64 neurona
cnn_adam.add(tf.keras.layers.Dense(64, activation='relu'))
cnn_adam.add(tf.keras.layers.Dense(64, activation='relu'))

cnn_adam.add(tf.keras.layers.Dense(1, activation='sigmoid'))
cnn_adam.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
start = time.time()
adam_history = cnn_adam.fit(x=X_train, epochs=20, validation_data = x_test,)
adam_time = time.time()-start

In [ ]:
metrike([cnn_adam], x_test, x_test.classes, [adam_time])

In [ ]:
plt.plot(pd.DataFrame(adam_history.history["accuracy"]))
plt.plot(pd.DataFrame(adam_history.history["val_accuracy"]))
plt.figure(figsize=(6,6), dpi=500)
plt.show()

plt.plot(pd.DataFrame(adam_history.history["loss"]))
plt.plot(pd.DataFrame(adam_history.history["val_loss"]))
plt.figure(figsize=(6,6), dpi=500)
plt.show()

## CNN - ADAMAX

In [ ]:
cnn_adamax = tf.keras.Sequential()
cnn_adamax.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
cnn_adamax.add(tf.keras.layers.MaxPooling2D((2, 2)))
cnn_adamax.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
cnn_adamax.add(tf.keras.layers.MaxPooling2D((2, 2)))
cnn_adamax.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
cnn_adamax.add(tf.keras.layers.MaxPooling2D((2, 2)))
cnn_adamax.add(tf.keras.layers.Flatten())
cnn_adamax.add(tf.keras.layers.Dense(64, activation='relu'))
cnn_adamax.add(tf.keras.layers.Dense(64, activation='relu'))
cnn_adamax.add(tf.keras.layers.Dense(1, activation='sigmoid'))
cnn_adamax.compile(optimizer='adamax',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
start = time.time()
adamax_history = cnn_adamax.fit(x=X_train, epochs=25,validation_data = x_test,)
adamax_time = time.time()-start

In [ ]:
metrike([cnn_adamax], x_test, x_test.classes, [adamax_time])

In [ ]:
plt.plot(pd.DataFrame(adamax_history.history["accuracy"]))
plt.plot(pd.DataFrame(adamax_history.history["val_accuracy"]))
plt.figure(figsize=(6,6), dpi=500)
plt.show()

plt.plot(pd.DataFrame(adamax_history.history["loss"]))
plt.plot(pd.DataFrame(adamax_history.history["val_loss"]))
plt.figure(figsize=(6,6), dpi=500)
plt.show()

## CNN - ADAGRAD

In [ ]:
cnn_adagrad = tf.keras.Sequential()
cnn_adagrad.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
cnn_adagrad.add(tf.keras.layers.MaxPooling2D((2, 2)))
cnn_adagrad.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
cnn_adagrad.add(tf.keras.layers.MaxPooling2D((2, 2)))
cnn_adagrad.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
cnn_adagrad.add(tf.keras.layers.MaxPooling2D((2, 2)))
cnn_adagrad.add(tf.keras.layers.Flatten())
cnn_adagrad.add(tf.keras.layers.Dense(64, activation='relu'))
cnn_adagrad.add(tf.keras.layers.Dense(64, activation='relu'))
cnn_adagrad.add(tf.keras.layers.Dense(1, activation='sigmoid'))
cnn_adagrad.compile(optimizer='adagrad',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
start = time.time()
adagrad_history = cnn_adagrad.fit(x=X_train, epochs=50,validation_data = x_test,)
adagrad_time = time.time()-start

In [ ]:
metrike([cnn_adagrad], x_test, x_test.classes, [adagrad_time])

In [ ]:
plt.plot(pd.DataFrame(adagrad_history.history["accuracy"]))
plt.plot(pd.DataFrame(adagrad_history.history["val_accuracy"]))
plt.figure(figsize=(6,6), dpi=500)
plt.show()

plt.plot(pd.DataFrame(adagrad_history.history["loss"]))
plt.plot(pd.DataFrame(adagrad_history.history["val_loss"]))
plt.figure(figsize=(6,6), dpi=500)
plt.show()

## CNN - NADAM

In [ ]:
cnn_nadam = tf.keras.Sequential()
cnn_nadam.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
cnn_nadam.add(tf.keras.layers.MaxPooling2D((2, 2)))
cnn_nadam.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
cnn_nadam.add(tf.keras.layers.MaxPooling2D((2, 2)))
cnn_nadam.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
cnn_nadam.add(tf.keras.layers.MaxPooling2D((2, 2)))
cnn_nadam.add(tf.keras.layers.Flatten())
cnn_nadam.add(tf.keras.layers.Dense(64, activation='relu'))
cnn_nadam.add(tf.keras.layers.Dense(64, activation='relu'))
cnn_nadam.add(tf.keras.layers.Dense(1, activation='sigmoid'))
cnn_nadam.compile(optimizer='nadam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
start = time.time()
nadam_history = cnn_nadam.fit(x=X_train, epochs=20,validation_data = x_test,)
nadam_time = time.time()-start

In [ ]:
metrike([cnn_nadam], x_test, x_test.classes, [nadam_time])

In [ ]:
plt.plot(pd.DataFrame(nadam_history.history["accuracy"]))
plt.plot(pd.DataFrame(nadam_history.history["val_accuracy"]))
plt.figure(figsize=(6,6), dpi=500)
plt.show()

plt.plot(pd.DataFrame(nadam_history.history["loss"]))
plt.plot(pd.DataFrame(nadam_history.history["val_loss"]))
plt.figure(figsize=(6,6), dpi=500)
plt.show()

## CNN - SGD

In [ ]:
cnn_sgd = tf.keras.Sequential()
cnn_sgd.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
cnn_sgd.add(tf.keras.layers.MaxPooling2D((2, 2)))
cnn_sgd.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
cnn_sgd.add(tf.keras.layers.MaxPooling2D((2, 2)))
cnn_sgd.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
cnn_sgd.add(tf.keras.layers.MaxPooling2D((2, 2)))
cnn_sgd.add(tf.keras.layers.Flatten())
cnn_sgd.add(tf.keras.layers.Dense(64, activation='relu'))
cnn_sgd.add(tf.keras.layers.Dense(64, activation='relu'))
cnn_sgd.add(tf.keras.layers.Dense(1, activation='sigmoid'))
cnn_sgd.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
start = time.time()
sgd_history = cnn_sgd.fit(x=X_train, epochs=40,validation_data = x_test,)
sgd_time = time.time()-start

In [ ]:
metrike([cnn_sgd], x_test, x_test.classes, [sgd_time])

In [ ]:
plt.plot(pd.DataFrame(sgd_history.history["accuracy"]))
plt.plot(pd.DataFrame(sgd_history.history["val_accuracy"]))
plt.figure(figsize=(6,6), dpi=500)
plt.show()

plt.plot(pd.DataFrame(sgd_history.history["loss"]))
plt.plot(pd.DataFrame(sgd_history.history["val_loss"]))
plt.figure(figsize=(6,6), dpi=500)
plt.show()

## CNN - RMSprop

In [ ]:
cnn_rms = tf.keras.Sequential()
cnn_rms.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
cnn_rms.add(tf.keras.layers.MaxPooling2D((2, 2)))
cnn_rms.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
cnn_rms.add(tf.keras.layers.MaxPooling2D((2, 2)))
cnn_rms.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
cnn_rms.add(tf.keras.layers.MaxPooling2D((2, 2)))
cnn_rms.add(tf.keras.layers.Flatten())
cnn_rms.add(tf.keras.layers.Dense(64, activation='relu'))
cnn_rms.add(tf.keras.layers.Dense(64, activation='relu'))
cnn_rms.add(tf.keras.layers.Dense(1, activation='sigmoid'))
cnn_rms.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
start = time.time()
rms_history = cnn_rms.fit(x=X_train, epochs=30,validation_data = x_test,)
rms_time = time.time()-start

In [ ]:
metrike([cnn_rms], x_test, x_test.classes, [rms_time])

In [ ]:
plt.plot(pd.DataFrame(rms_history.history["accuracy"]))
plt.plot(pd.DataFrame(rms_history.history["val_accuracy"]))
plt.figure(figsize=(6,6), dpi=500)
plt.show()

plt.plot(pd.DataFrame(rms_history.history["loss"]))
plt.plot(pd.DataFrame(rms_history.history["val_loss"]))
plt.figure(figsize=(6,6), dpi=500)
plt.show()

In [ ]:
models = [cnn_adam,cnn_adamax,cnn_adagrad,cnn_nadam,cnn_sgd,cnn_rms]
times = [adam_time,adamax_time,adagrad_time, nadam_time, sgd_time,rms_time]
histories = [adam_history,adamax_history,adagrad_history,nadam_history,sgd_history,rms_history]
metrike_table(models, x_test, x_test.classes, times, histories)

In [ ]:
for model in models:
    model.save(f'../saved_models/animal_classification/{type(model.optimizer).__name__}')